Imports

In [1212]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
from plotting import plot_array
from soundGen import play
from pointsFrequency import signal_to_hertz
from filterAudio import low_pass_Filter, high_pass_Filter, weierstrassFunc,\
     dirac_comb_discrete, Rev_Conv_Filter
import torch
from torch_pitch_shift import pitch_shift
from numpy.random import uniform


Set universal variables above functions and declare them 

In [1213]:
Fs = 44100
T = 1/(Fs) 
N = Fs
amplitude = 2
frequency = [400, 200, 100]
t = 1
omega = 2 * np.pi


Then create code for signal

In [1214]:
t_vec = np.arange(N)*T*t 
ysum = 0
sine_add = np.sin(np.linspace(-16 * np.pi,16 * np.pi,len(t_vec)))/10
omega = 2 * np.pi* sine_add
for i in range(len(frequency)):
    ysum  += amplitude * np.sin(omega * frequency[i] * t_vec)


Normalize the signal


In [1215]:
normalized_y = ysum/ np.max(ysum)

rev_y = Rev_Conv_Filter(normalized_y.copy(), 2, 100)
plt.plot()
play(rev_y)

UnboundLocalError: local variable 'releaseg' referenced before assignment